In [34]:
import pandas as pd
import numpy as np
import torch
from utils import parse_answers, verify_answer, evaluate_responses

In [2]:
TRAIN_ACTIVATIONS_PATH="activations/qwen2_test.pt"

In [3]:
train_data = torch.load(TRAIN_ACTIVATIONS_PATH)

In [5]:
train_activations = train_data['activations']  # [N, L, P, D]
train_labels = train_data['labels'].numpy() 

In [7]:
# (n_prompt, n_layer, n_pos, hidden_size)
train_activations.shape

torch.Size([500, 29, 5, 1536])

In [73]:
THOMS_MATH_DATA = "../predicting_learnability/data/SR_DATASETS/THOMS_MATH"

TRAIN_PATH = f"{THOMS_MATH_DATA}/MATH_train-Qwen-Qwen2-1.5B-Instruct_lorem.parquet"
TEST_PATH = f"{THOMS_MATH_DATA}/MATH_test-Qwen-Qwen2-1.5B-Instruct_lorem.parquet"


# TRAIN_PATH = "../runs/Qwen2-1.5B-Instruct/datasplits/predicting_MATH_learnability_Qwen2-1.5B-Instruct_max_3000_k_1_temp_0.0_train.json"
# TEST_PATH = "../runs/Qwen2-1.5B-Instruct/datasplits/predicting_MATH_learnability_Qwen2-1.5B-Instruct_max_3000_k_1_temp_0.0_test.json"

MATH_DATA = "DATA/SR_DATA/MATH" #"SR_DATA"

TRAIN_PATH = f"{MATH_DATA}/train-Qwen2-1.5B-Instruct_maxlen_3000_k_0_temp_0.0.parquet"
TEST_PATH = f"{MATH_DATA}/test-Qwen2-1.5B-Instruct_maxlen_3000_k_0_temp_0.0.parquet"


TRAIN_DF = pd.read_parquet(TRAIN_PATH)
TEST_DF = pd.read_parquet(TEST_PATH)
# TRAIN_DF = pd.read_json(TRAIN_PATH)
# TEST_DF = pd.read_json(TEST_PATH)

In [74]:
len(TRAIN_DF)

7500

In [75]:
len(TEST_DF)

5000

In [77]:
TEST_DF

,generated_solutions,ground_truth,formatted_prompt,problem,success_rate
0,"[{'score': 1.0, 'text': ' To find the vertical...",2,<|im_start|>system\nYou are a helpful assistan...,How many vertical asymptotes does the graph of...,1.0
1,"[{'score': 1.0, 'text': ' First, we calculate ...",10,<|im_start|>system\nYou are a helpful assistan...,What is the positive difference between $120\%...,1.0
2,"[{'score': 0.0, 'text': 'To find $x$ such that...",\dfrac{9}{7},<|im_start|>system\nYou are a helpful assistan...,Find $x$ such that $\lceil x \rceil + x = \dfr...,0.0
3,"[{'score': 0.0, 'text': ' First, we need to un...",i,<|im_start|>system\nYou are a helpful assistan...,Evaluate $i^5+i^{-25}+i^{45}$.,0.0
4,"[{'score': 1.0, 'text': ' We can rewrite $4^x$...",4,<|im_start|>system\nYou are a helpful assistan...,"If $2^8=4^x$, what is the value of $x$?",1.0
...,...,...,...,...,...
4995,"[{'score': 0.0, 'text': 'Given $\sin x + \cos ...",-\frac{4}{3},<|im_start|>system\nYou are a helpful assistan...,If $\sin x + \cos x = \frac{1}{5}$ and $0 < x ...,0.0
4996,"[{'score': 0.0, 'text': ' The normal vector of...",\begin{pmatrix} 1 \\ 2 \\ -3 \end{pmatrix},<|im_start|>system\nYou are a helpful assistan...,The matrix for projecting onto a certain plane...,0.0
4997,"[{'score': 1.0, 'text': ' From the given equat...",-\frac{155}{2},<|im_start|>system\nYou are a helpful assistan...,"Let $\mathbf{a},$ $\mathbf{b},$ and $\mathbf{c...",1.0
4998,"[{'score': 0.0, 'text': ' First, we simplify t...",159,<|im_start|>system\nYou are a helpful assistan...,Find the smallest positive integer solution to...,0.0


In [76]:
TEST_DF["formatted_prompt"].iloc[0]

"<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\nHow many vertical asymptotes does the graph of $y=\\frac{2}{x^2+x-6}$ have? Let's think step by step and output the final answer within \\boxed{}.<|im_end|>\n<|im_start|>assistant\n"

In [ ]:
# TRAIN_DF["generated_solutions"].iloc[0][0]

{'score': 0.0,
 'text': " We need to ensure that the function defined by the piecewise rule is continuous at $x = 2$. Continuity at $x = 2$ means that the two sides of the piecewise function must approach the same value as $x$ approaches 2 from the left and from the right.\n\nFor the first part:\n$$f(x) = ax + 3$$\nAt $x > 2$, as $x$ approaches 2 from the right ($x=2+0.001$), we get:\n$$f(x) = a(2+0.001) + 3 = 2a + 0.001 + 3 = 2a + 3.001$$\nFor $x > 2$, as $x$ approaches 2 from the left ($x=2-0.001$), we get:\n$$f(x) = a(2-0.001) + 3 = 2a - 0.001 + 3 = 2a + 2.999$$\nSince both sided approaches $f(x) = 2a + 3.001$ as $x$ approaches 2, the function is continuous.\n\nFor the second part:\n$$f(x) = x - 5$$\nAt $-2 \\le x \\le 2$, as $x$ approaches 2 from the left of 2, we get:\n$$f(x) = -2$$\nAnd as $x$ approaches 2 from the right of 2, we get:\n$$f(x) = -4$$\nSince both sides approach $f(x) = -4$, the function is continuous as well.\n\nFor the third part:\n$$f(x) = 2x - b$$\nAt $x <-2$, a

In [22]:
print(TRAIN_DF["question"].iloc[0])

<|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
How many vertical asymptotes does the graph of $y=\frac{2}{x^2+x-6}$ have? Let's think step by step and output the final answer after \boxed{}.<|im_end|>
<|im_start|>assistant

